<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
# !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [4]:
import os
os.chdir(r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix/')

In [3]:
!pip install -r requirements.txt

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

### First check if dataset name is same for A-B pix2pix pair: (pix2pix requires same image pair)

In [58]:
import os
import random
import shutil
from glob import glob
import os
import pandas as pd

# Set up the paths for directories A and B
dir_a = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE'
dir_b = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\Unstained'
dir_a = glob(os.path.join(dir_a,'*','*.png'))
dir_b = glob(os.path.join(dir_b,'*','*.png'))
dir_a = [os.path.basename(x) for x in dir_a]
dir_b = [os.path.basename(x) for x in dir_b]
dir_a = [x.split('xy')[1] for x in dir_a] # split on xy to get the image #
dir_b = [x.split('xy')[1] for x in dir_b] # split on xy to get the image #

split_a_images = set(dir_a)
split_b_images = set(dir_b)
only_in_a = split_a_images - split_b_images
only_in_b = split_b_images - split_a_images

print(len(only_in_a))
print(len(only_in_b))

0
0


### Dataset creation for HE (B), filter >70% whitespace images and then randomly shuffle images and move them to train/test/val split local folder:

In [99]:
# Set the seed for random shuffle
from tqdm import tqdm
import math
random.seed(42) # repeatability

hepath = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE'
all_he_path = glob(os.path.join(hepath,'*','*.png')) # list of all images
nspath = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\Unstained'
all_ns_path = glob(os.path.join(nspath,'*','*.png')) # list of all images

train_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\train'
valid_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\val'
test_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\test'

if not os.path.exists(train_path):
    os.makedirs(train_path)

if not os.path.exists(valid_path):
    os.makedirs(valid_path)

if not os.path.exists(test_path):
    os.makedirs(test_path)

# filter all_he_path:
comp_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\composition\pkl'
unstained_path = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\Unstained'
desired_he_path = []
desired_ns_path = []
for idx in tqdm(range(len(all_he_path)),total=len(all_he_path),desc='Images Processed',colour='red'):
    #extract image name:
    hename = all_he_path[idx]
    imgname = hename.split("HE")[1].split("\\")[1]
    tilename = os.path.basename(hename)
    tilename = tilename[:-4]
    pkl_src = os.path.join(comp_src,imgname + ".pkl")
    if not os.path.exists(pkl_src):
        continue #images where compositoin doesn't exist, can't use
    pkl = pd.read_pickle(pkl_src)
    comp_df = pkl[tilename]
    if comp_df["white"] > 70 and not math.isnan(comp_df["white"]): # if >70% whitespace and whitespace is not nan, it is a background white image, so drop.
        continue
    else:
        desired_he_path.append(hename)
        nsname = all_ns_path[idx]
        desired_ns_path.append(nsname)
        # ns_imgname = imgname[:-3] # drop "_he"
        # imgnumber = tilename.split("xy")[1].split(".png")[0]
        # ns_equivalent_path = os.path.join(unstained_path,ns_imgname)
        # ns_img_pair = [x for x in ns_equivalent_path if imgnumber in x.split('xy')[1]] # search by the image index
        # desired_ns_path.append(ns_img_pair)




Images Processed: 100%|██████████| 274647/274647 [9:45:52<00:00,  7.81it/s]    


In [100]:
desired_he_path
print(len(desired_he_path))

79271


In [101]:
desired_ns_path
print(len(desired_ns_path))

79271


In [102]:
desired_he_path

['\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\8294_45158xy0688.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\8294_46182xy0689.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\8294_47206xy0690.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\8294_48230xy0691.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\8294_49254xy0692.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\8294_50278xy0693.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\8294_51302xy0694.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\9318_44134xy0779.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\9318_45158xy0780.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\

In [104]:
df_src = r"\\shelter\Kyu\unstain2stain\tiles\registrated_tiles"
he_df = pd.DataFrame(desired_he_path)
he_df.to_excel(os.path.join(df_src,'he_train_image_path.xlsx'))
ns_df = pd.DataFrame(desired_ns_path)
ns_df.to_excel(os.path.join(df_src,'ns_train_image_path.xlsx'))

In [57]:
#80-10-10 split
train_pct = 0.8
valid_pct = 0.1
test_pct = 0.1

random.shuffle(all_he_path)
image_files = [os.path.basename(x) for x in all_he_path] # ex 1124_1124xy0001.png basename of image
img_type = [x.split("HE")[1].split("\\")[1] for x in all_he_path] #ex. OTS_14684_3. name of image so we can know what image it's from

num_files = len(image_files)
num_train = int(num_files * train_pct)
num_valid = int(num_files * valid_pct)
num_test = int(num_files * test_pct)

# Copy the image files into the train folder
for i in tqdm(range(num_train),total=num_train, colour='red',desc = 'Files Processed'):
    src_path = all_he_path[i]
    dst_path = os.path.join(train_path, img_type[i] + "_" + image_files[i])
    shutil.copyfile(src_path, dst_path)

# Copy the image files into the validation folder
for i in tqdm(range(num_train, num_train+num_valid),total=num_train+num_valid-num_train, colour='red',desc = 'Files Processed'):
    src_path = all_he_path[i]
    dst_path = os.path.join(valid_path, img_type[i] + "_" + image_files[i])
    shutil.copyfile(src_path, dst_path)

# Copy the image files into the test folder
for i in tqdm(range(num_train+num_valid, num_files),total=num_files-(num_train+num_valid),colour='red',desc = 'Files Processed'):
    src_path = all_he_path[i]
    dst_path = os.path.join(test_path, img_type[i] + "_" + image_files[i])
    shutil.copyfile(src_path, dst_path)


['\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_5_he\\82022_65638xy8596.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14832_4_he\\95334_12390xy8936.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_8_he\\13414_36966xy1140.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14832_14_he\\33894_79974xy2574.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14832_11_he\\109670_61542xy11508.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14832_13_he\\76902_2150xy7180.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\65638_52326xy5847.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14832_9_he\\57446_15462xy4305.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14832_15_he\\8294_3174xy0647.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrate

In [5]:
import os
import shutil



parent_dir = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B'
new_images_dir = all_ns_path
new_save_dir = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A'

# Loop through each original directory (train, val, test)
for split_dir in os.listdir(parent_dir): # train,val,test
    split_path = os.path.join(parent_dir, split_dir)

    # Get the set of image file names for this split
    split_images = set(os.listdir(split_path)) # train,val,test image names
    # Loop through each image in the new images directory
    for image_file in os.listdir(new_images_dir):
        # Check if the image is in the current split
        if image_file in split_images:
            image_path = os.path.join(new_images_dir, image_file)
            new_split_path = os.path.join(new_save_dir, split_dir, image_file)
            shutil.copyfile(image_path, new_split_path)


### Final check to see if all images moved to A and B are the same:

In [1]:
import os

dir_a = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A'
dir_b = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B'

# Loop through each split (train, val, test)
for split in ['train', 'val', 'test']:
    split_a_path = os.path.join(dir_a, split)
    split_b_path = os.path.join(dir_b, split)

    # Get the set of image file names for each split in directory A
    split_a_images = set(os.listdir(split_a_path))

    # Get the set of image file names for each split in directory B
    split_b_images = set(os.listdir(split_b_path))

    # Find the set of images that are in directory A but not in directory B
    only_in_a = split_a_images - split_b_images

    # Find the set of images that are in directory B but not in directory A
    only_in_b = split_b_images - split_a_images

    # Print the results
    print(f"Split: {split}")
    print(f"Only in {dir_a}: {len(only_in_a)}")
    print(f"Only in {dir_b}: {len(only_in_b)}")
    print("-----------")


Split: train
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------
Split: val
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------
Split: test
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------


## Run this to finish generating dataset:

In [3]:
!python datasets/combine_A_and_B.py --fold_A C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A --fold_B C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B --fold_AB C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --no_multiprocessing

[fold_A] =  C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A
[fold_B] =  C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B
[fold_AB] =  C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 357/357 images
split = test, number of images = 357
split = train, use 2856/2856 images
split = train, number of images = 2856
split = val, use 357/357 images
split = val, number of images = 357


# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
# !bash ./scripts/download_pix2pix_model.sh facades_label2photo

In [4]:
import torch
torch.cuda.is_available()

True

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transform from class B to A.

In [1]:
!python train.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --name unstain2stain_pix2pix --model pix2pix --direction AtoB --display_id -1 --display_freq 400 --display_ncols 4 --update_html_freq 1000 --print_freq 100 --checkpoints_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\checkpoints --save_epoch_freq 5 --n_epochs 1000

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
# !ls checkpoints/

In [ ]:
!python datasets/combine_A_and_B.py --fold_A C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A --fold_B C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B --fold_AB C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --no_multiprocessing

In [2]:
!python test.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --direction AtoB --model pix2pix --name unstain2stain_pix2pix --results_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results --num_test 50

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
          

### Inference for proof-of-concept: Tiled original unstained image -> run inference -> Output stained image tiles -> Reconstruct to fake WSI HE Image

### Make A the registered unstained tiles and B the registered HE tiles from the matlab registration code from WSI then run inference

In [21]:
!python test.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --model pix2pix --name unstain2stain_pix2pix --results_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results --direction AtoB --num_test 10995 --load_size 1024 --crop_size 1024

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 1024                          	[default: 256]
                 dataroot: C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: N

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)